In [1]:
from scipy.io import mmread
import os
import pandas as pd
import numpy as np
 

## Load gene and peak count data

In [2]:
# import scRNA-seq gene count data matrix and scATAC-seq peak count data matrix
# NOTE: the original file is a .txt file in the MatrixMarket format
# I just changed the .txt extension to .mtx 
os.chdir('C:/Users/lexie/Documents/SMU/super-test/paper-code/scGCN_tensorflow2/A549/')
rna = mmread('GSM3271040_RNA_sciCAR_A549_gene_countMAT.mtx')
atac = mmread('GSM3271041_ATAC_sciCAR_A549_peak_countMAT.mtx')
display(atac)
display(rna)

<189603x6085 sparse matrix of type '<class 'numpy.float64'>'
	with 1950459 stored elements in COOrdinate format>

<113153x6093 sparse matrix of type '<class 'numpy.int64'>'
	with 9251101 stored elements in COOrdinate format>

In [3]:
# convert RNA and ATAC sparse matrix to dense matrix
rna = rna.todense()
atac = atac.todense()

MemoryError: Unable to allocate 8.60 GiB for an array with shape (189603, 6085) and data type float64

In [ ]:
# check dimensions of RNA and ATAC data matrix
# columns are cells
print('Shape of RNA data matrix: ',rna.shape)
print('Shape of ATAC data matrix: ',atac.shape)

## Load scRNA-seq cell labels

In [ ]:
# import RNA-seq cell labels
# rows are cells
RNAcell_lab = pd.read_csv('GSM3271040_RNA_sciCAR_A549_cell.txt', sep = ',')
display(RNAcell_lab)

## Filter out non-A549 cells in scRNA-seq data

In [ ]:
# get the index of the cells that are A549
include = RNAcell_lab.index[[RNAcell_lab['cell_name'] == 'A549']].tolist()

# only keep the column labels corresponding to A549 cells
RNAcell_lab = RNAcell_lab.iloc[include]
print('New shape of RNA cell label dataframe: ',RNAcell_lab.shape)
display(RNAcell_lab)

In [ ]:
# only keep the RNA column data correponding to gene counts for A549 cells
# rows are genes
# columns are cells
rna = rna[:,include]
print('New shape of RNA data matrix: ',rna.shape)

## Cross-reference cell names between scRNA-seq and scATAC-seq

In [ ]:
# import ATAC-seq data and cell labels
ATACcell_lab = pd.read_csv('ATAC_cells_reduced.csv')
display(ATACcell_lab)

In [ ]:
# only keep cells that are common to both RNA-seq data and ATAC-seq data
# since RNA data has more cells, loop through cells and make sure it has a counterpart in the ATAC cell labels

match = []
atac_match = []
for cell in range(RNAcell_lab.shape[0]):
    thiscell = RNAcell_lab.iloc[cell,:]
    
    if thiscell['sample'] in list(ATACcell_lab['sample']):
        match.append(cell)
        
        # find the index of this cell in the ATAC data
        atac_match.append(ATACcell_lab.index[ATACcell_lab['sample']==thiscell['sample']].to_list())
        
    else:
        continue

# if there is no match between RNA and ATAC cell label, remove that row and corresponding column in the RNA data
RNAcell_red = RNAcell_lab.iloc[match]
print('New shape of RNA cell dataframe: ',RNAcell_red.shape)

atac_match = np.concatenate(atac_match)
print(atac_match)

In [ ]:
# only keep the column data correponding to gene counts for matched cells
rna = rna[:,match]
print('New shape of RNA data matrix: ',rna.shape)
type(rna)

In [ ]:
# create new ATAC cell and data matrices based on the corresponding RNA cells
# use the indices found from atac_match 
# in other words, only keep cells that are common to both RNA-seq data and ATAC-seq data

ATACcell_red = ATACcell_lab.iloc[atac_match]
display(ATACcell_red)
print('New shape of ATAC cell label dataframe: ',ATACcell_red.shape)

atac = atac[:,atac_match]
print('\n New shape of ATAC data matrix: ',atac.shape)
type(atac)


In [ ]:
# sanity check: 
# make sure sample names in ATAC and RNA cells are the same for each row
print(ATACcell_red['sample'].to_list() == RNAcell_red['sample'].to_list())

# spot check
print(ATACcell_red['sample'].to_list()[120])
print(RNAcell_red['sample'].to_list()[120])

# save out reduced cell labels
# reduced version only includes matched cells
ATACcell_red.to_csv('ATAC_cells_reduced.csv')
RNAcell_red.to_csv('RNA_cells_reduced.csv')

In [ ]:
# again double-check that the treatment times are the same for both datasets
alabels = ATACcell_red['treatment_time'].to_list()
rlabels = RNAcell_red['treatment_time'].to_list()
print(alabels == rlabels)


# save a list of the treatment time for each cell
# reference is first
labellist = [rlabels,alabels]
print(len(labellist[0]))
print(len(labellist[1]))

# np.savetxt('label_list.csv', labellist, delimiter=',')

In [ ]:
ATACcell_red['sample']

In [ ]:
# save out cell names
#np.savetxt('cell_list.csv', list(ATACcell_red['sample']), delimiter=',', fmt = '%s')

## Cross-reference gene and peak labels

In [ ]:
# import RNA-seq genes
RNAgene_lab = pd.read_csv('GSM3271040_RNA_sciCAR_A549_gene.txt', sep = ',')
display(RNAgene_lab)

In [ ]:
# import annotated ATAC-seq peaks
ATACpeak_lab = pd.read_csv('ATAC_peaks_data_annot.csv', sep = ',')
display(ATACpeak_lab)

In [ ]:
# cross-reference gene names
# of the gene labels in the RNA data, find which are also in the ATAC data

# get the original index of the gene names
ind_dict = dict((k,i) for i,k in enumerate(RNAgene_lab['gene_short_name']))

# find the common gene names
gene_match = set(RNAgene_lab['gene_short_name']).intersection(ATACpeak_lab['gene'])
print('Number of gene matches: ',len(gene_match))

# find the indices of the common genes in the original RNAgene_red 
keep = [ind_dict[x] for x in gene_match]

# only keep the genes that are in both the RNA and ATAC data
RNAgene_red = RNAgene_lab.iloc[keep]
print('\n New shape of RNA gene label dataframe: ', RNAgene_red.shape)

rna = rna[keep,:]
print('\n New shape of RNA data matrix: ', rna.shape)

In [ ]:
# save out reduced RNA-seq gene labels reduced version only includes matched genes
# RNAgene_red.to_csv('RNA_genes_reduced.csv')

In [ ]:
# save out reduced RNA data
# reduced version only includes matched genes and matched cells
# np.savetxt("RNA_data_reduced.csv", rna, delimiter=",")

In [ ]:
# save out gene short names
# np.savetxt("gene_list.csv", RNAgene_red['gene_short_name'], delimiter=",", fmt = '%s')

In [ ]:
# check for duplicates in ATAC peak labels

# find how many duplicate annotated genes are in the ATAC seq gene labels
rep = sum(ATACpeak_lab.gene.duplicated())
print('Number of duplicate annotated genes in ATAC peak labels: ', rep)

# sanity check: how many unique items
uni = len(set(ATACpeak_lab['gene']))
print('\n Number of unique items: ', uni)

print('\n Sum = ',rep + uni)
rep + uni == ATACpeak_lab.shape[0]

In [ ]:
# find how many duplicate annotated genes are in the RNA seq gene labels
# there should be 0

rep = sum(RNAgene_red.gene_short_name.duplicated())
print('Number of duplicate genes in RNA seq gene labels: ', rep)

## Convert scATAC-seq data to gene-based activity

In [ ]:
# for each gene in the RNA data, find entries for that gene in ATAC peaks
# if there are multiple entries for a gene in ATAC data, take the sum as the value for the gene count
# save these values in a new matrix as the new ATAC data

atac_match_sum = []
atac_match_idx = []
for gene in range(RNAgene_red.shape[0]):
    thisgene = RNAgene_red.iloc[gene,:]
    matches = []
    
    # find entries in the ATAC labels that match this gene
    # note: there may be multiple
    matches = ATACpeak_lab.index[ATACpeak_lab['gene'] == thisgene['gene_short_name']].tolist()
    
    # if this gene in the RNA data has a match or matches in the ATAC data, 
    # create a new dataframe with the ATAC information
    if matches:
        # get the corresponding count data for the matches
        atac_match_idx.append(matches)
        atac_match_sum.append(atac[matches,:].sum(axis=0))

print(atac_match_idx)
print('\n Number of gene matches: ', len(atac_match_idx))
print('\n Number of rows in atac match sum: ', len(atac_match_sum))
print('\n Number of columns in atac match sum: ', atac_match_sum[0].shape[1])
atac_red = np.stack(atac_match_sum, axis=0)
#atac_red

In [ ]:
# save out reduced ATAC data
# reduced version only includes matched genes and matched cells
# np.savetxt("ATAC_data_reduced.csv", atac_red, delimiter=",")

In [ ]:
# double-check the shapes of the RNA and ATAC data
# they should be the same
print(atac_red.shape)
print(rna.shape)
